In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [5]:
from langchain_groq import ChatGroq
model = ChatGroq(model="gemma2-9b-it", api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023D5BF66710>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023D5BF64220>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that helps people find information."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage
chain.invoke({"messages": [
    HumanMessage(content="Hi, my name is Dhananjai. I'm an AI Engineer")
]})

AIMessage(content="Hello Dhananjai, it's nice to meet you! \n\nAs an AI Engineer, I'm sure you're always looking for new information and ways to improve your work. How can I help you today? \n\nDo you have a specific question, or are you just browsing? I can help with things like:\n\n* **Finding research papers and articles** on specific AI topics\n* **Explaining complex AI concepts** in a simpler way\n* **Comparing different AI tools and frameworks**\n* **Generating code snippets** for common AI tasks\n\n\nLet me know what's on your mind! 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 132, 'prompt_tokens': 35, 'total_tokens': 167, 'completion_time': 0.24, 'prompt_time': 0.00148516, 'queue_time': 0.253310128, 'total_time': 0.24148516}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--83509075-8df7-4359-af04-0aa464da6d1c-0', usage_metadata={'input

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory


store={}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [10]:
config = {"configurable": {"session_id": "chat1"}}
response = with_message_history.invoke([
    HumanMessage(content="Hi, my name is Dhananjai. I'm an AI Engineer")
], config=config)

response.content

"Hello Dhananjai! It's nice to meet you. \n\nBeing an AI Engineer is fascinating. What kind of projects are you working on these days?  \n\nI'm here to help with any information you need. Just ask! 😊  \n\n"

In [11]:
# Add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions in this language: {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [13]:
response = chain.invoke({"messages": [
    HumanMessage(content="Hi, my name is Dhananjai. I'm an AI Engineer")
], "language": "Hindi"})

response.content

'नमस्ते धनंजय, मैं आपकी मदद करने के लिए यहाँ हूँ।  \n'

Let's now wrap this more complicated chain in a Message History Class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history

In [15]:
with_message_history = RunnableWithMessageHistory(
    chain, get_session_history, input_messages_key="messages"
)

config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke({
    "messages": [HumanMessage(content="Hi, my name is Dhananjai. I'm an AI Engineer")],
    "language": "Spanish"
}, config=config)

response.content

'¡Hola Dhananjai! Es un placer conocerte. Soy un asistente útil y estoy aquí para ayudarte con lo que necesites. 😊 ¿En qué te puedo ayudar?  \n\n'

Managing the Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of messages you are passing

In [16]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(max_tokens=70, strategy="last", token_counter=model, include_system=True, allow_partial=False, start_on="human")

🔧 Parameters Explained


max_tokens=70
Limits the total number of tokens in the trimmed message list to 70.


strategy="last"
This strategy keeps the most recent messages and discards older ones until the token limit is met. Other strategies might include "first", "middle", etc.


token_counter=model
This is a function or object that knows how to count tokens (likely your language model). It’s used to calculate how many tokens each message consumes.


include_system=True
Ensures that system messages (like instructions or context-setting messages) are included in the trimming process.


allow_partial=False
If set to False, it will not include partially trimmed messages. Only full messages that fit within the token limit are kept.


start_on="human"
Trimming will start from the last human message and work backwards. This is useful when you want to prioritize user inputs.

In [21]:
messages = [
    SystemMessage(content="You are a helpful assistant that helps people find information."),
    HumanMessage(content="Hi, my name is Dhananjai. I'm an AI Engineer"),
    SystemMessage(content="Hello Dhananjai! How can I assist you today?"),
    HumanMessage(content="Can you add 2+2?"),
    SystemMessage(content="Sure! It's 4"),
    HumanMessage(content="What's my favourite color?"),
    SystemMessage(content="I like green"),
    HumanMessage(content="Awesome! Thanks for the info."),
    SystemMessage(content="You're welcome! If you have any more questions, feel free to ask.")
]

trimmer.invoke(messages)

[SystemMessage(content='You are a helpful assistant that helps people find information.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Can you add 2+2?', additional_kwargs={}, response_metadata={}),
 SystemMessage(content="Sure! It's 4", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's my favourite color?", additional_kwargs={}, response_metadata={}),
 SystemMessage(content='I like green', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Awesome! Thanks for the info.', additional_kwargs={}, response_metadata={}),
 SystemMessage(content="You're welcome! If you have any more questions, feel free to ask.", additional_kwargs={}, response_metadata={})]

In [22]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer) 
    | prompt
    | model
)

response = chain.invoke({"messages": messages + [HumanMessage(content="What is my name?")],
             "language": "English"})

response.content

"As an AI, I don't have access to personal information about you, including your name.  \n\nIs there anything else I can help you with? 😊  \n"

In [23]:
response = chain.invoke({"messages": messages + [HumanMessage(content="What is my favourite color?")],
             "language": "English"})

response.content

'You told me your favorite color is green!  😊 \n'

In [25]:
#Let's now wrap this more complicated chain in a Message History Class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history

with_message_history = RunnableWithMessageHistory(
    chain, get_session_history, input_messages_key="messages"
)

config = {"configurable": {"session_id": "chat4"}}
response = with_message_history.invoke({"messages": [
    HumanMessage(content="Hi,What is my favourite color?")
], "language": "Hindi"}, config=config)

response.content

'मुझे नहीं पता कि आपका पसंदीदा रंग क्या है।  \n\n'